In [1]:
import pandas as pd
import os
from easydict import EasyDict
def get_parent(path=os.getcwd(), levels=1):
    common = path
    # Using for loop for getting starting point required for
    for i in range(levels + 1):
        # Starting point
        common = os.path.dirname(common)
    return os.path.abspath(common)
data_folder_path = os.path.join(get_parent(), "data")
res_dfs = []
dfs = []
results = []
result_df = pd.DataFrame()
daily_prices = pd.read_csv(os.path.join(data_folder_path, "03_scratch_data", 'daily_pool_agg_results.csv'))
weekly_prices = pd.read_csv(os.path.join(data_folder_path, "03_scratch_data", 'weekly_pool_agg_results.csv'))
TEST_POOL_ADDR = ['0x11b815efb8f581194ae79006d24e0d814b7697f6',
             '0x4e68ccd3e89f51c3074ca5072bbac773960dfa36',
             '0x69d91b94f0aaf8e8a2586909fa77a5c2c89818d5',
             '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640',
             '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8',
             '0x99ac8ca7087fa4a2a1fb6357269965a2014abc35',
             '0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8',
             ]
POOL_ADDR = TEST_POOL_ADDR

pool_info_edict_list = [
    EasyDict({'pair_name': 'ETH-USDT', 'fee_tier': 500, 'decimal0': 18, 'decimal1': 6, 'base_token0': False}),
    EasyDict({'pair_name': 'ETH-USDT', 'fee_tier': 3000, 'decimal0': 18, 'decimal1': 6, 'base_token0': False}),
    EasyDict({'pair_name': 'HEX-USDC', 'fee_tier': 3000, 'decimal0': 8, 'decimal1': 6, 'base_token0': False}),
    EasyDict({'pair_name': 'USDC-ETH', 'fee_tier': 500, 'decimal0': 6, 'decimal1': 18, 'base_token0': True}),
    EasyDict({'pair_name': 'USDC-ETH', 'fee_tier': 3000, 'decimal0': 6, 'decimal1': 18, 'base_token0': True}),
    EasyDict({'pair_name': 'WBTC-USDC', 'fee_tier': 3000, 'decimal0': 8, 'decimal1': 6, 'base_token0': False}),
    EasyDict({'pair_name': 'DAI-ETH', 'fee_tier': 3000, 'decimal0': 18, 'decimal1': 18, 'base_token0': True}),
]



POOL_INFO = dict(zip(POOL_ADDR, pool_info_edict_list))
sc_from_data = pd.read_csv(os.path.join(data_folder_path, "01_original_data", "sc_ownership", 'token_transfer_sc_from_data.csv'))
sc_to_data = pd.read_csv(os.path.join(data_folder_path, "01_original_data", "sc_ownership", 'token_transfer_sc_to_data.csv'))
sc_original_to_data = pd.read_csv(os.path.join(data_folder_path, "01_original_data", "sc_ownership", 'token_transfer_sc_original_to_data.csv'))

STABLE_COIN_ADDR = [
    '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
    '0x6b175474e89094c44da98b954eedeac495271d0f',
    '0xdac17f958d2ee523a2206206994597c13d831ec7',
]

In [10]:
sc_all_data = pd.concat([sc_from_data, sc_to_data, sc_original_to_data])
sc_all_data.to_csv(os.path.join(data_folder_path, "03_scratch_data", 'token_transfer_sc_all_data.csv'), index=False)

In [9]:
sc_all_data_no_duplicates = sc_all_data = sc_all_data.drop_duplicates()
sc_all_data_no_duplicates.to_csv(os.path.join(data_folder_path, "03_scratch_data", 'token_transfer_sc_all_data_no_dups.csv'), index=False)

In [11]:
sc_verification_data = pd.read_csv("C:/Users/zxz170017/verification.csv")

In [17]:
sc_verification_data_unique_managers = sc_verification_data["manager_address"].unique()

In [15]:
sc_all_data_no_duplicates.head(10)["TX_HASH"][0]

'0x44e5e0e2054817e79c8b577c5ac2b46d1241a713a4f1491c413db81e48facc23'

In [31]:
token_decimals = pd.read_csv(os.path.join(data_folder_path, "01_original_data", "swap_data", "token_decimals.csv"))

In [34]:
sc_all_data_no_duplicates = sc_all_data_no_duplicates.merge(token_decimals, on="CONTRACT_ADDRESS", how="left").dropna()

In [37]:
# use gmpy2 to handle the precision issue
from gmpy2 import mpz
sc_all_data_no_duplicates["raw_amount_mpz"] = sc_all_data_no_duplicates["RAW_AMOUNT_PRECISE"].apply(lambda x: mpz(x))
sc_all_data_no_duplicates["amount"] = sc_all_data_no_duplicates["raw_amount_mpz"] / (10 ** sc_all_data_no_duplicates["DECIMAL"])


In [40]:
sc_all_data_no_duplicates.sort_values(by=["BLOCK_NUMBER", "TX_HASH", "EVENT_INDEX"], inplace=True)

In [43]:
sc_all_data_no_duplicates["week"] = pd.to_datetime(sc_all_data_no_duplicates["BLOCK_TIMESTAMP"]).dt.to_period('W-SAT').dt.start_time

In [46]:
# convert all columns to lower case string
sc_all_data_no_duplicates.columns = sc_all_data_no_duplicates.columns.str.lower()


In [65]:
sc_all_data_no_duplicates["contract_address"].unique()

array(['0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
       '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
       '0x6b175474e89094c44da98b954eedeac495271d0f',
       '0xdac17f958d2ee523a2206206994597c13d831ec7',
       '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599'], dtype=object)

In [51]:
sc_all_data_no_duplicates["is_to_pool"] = sc_all_data_no_duplicates["to_address"].isin(TEST_POOL_ADDR)

In [68]:
sc_all_data_no_duplicates["is_from_pool"] = sc_all_data_no_duplicates["from_address"].isin(TEST_POOL_ADDR)
sc_all_data_no_duplicates["is_stable_coin"] = sc_all_data_no_duplicates["contract_address"].isin(STABLE_COIN_ADDR)

In [84]:
to_pool_obs = sc_all_data_no_duplicates[sc_all_data_no_duplicates["is_to_pool"]].copy()
from_pool_obs = sc_all_data_no_duplicates[sc_all_data_no_duplicates["is_from_pool"]].copy()

In [85]:
neither_to_pool_nor_from_pool = sc_all_data_no_duplicates[~sc_all_data_no_duplicates["is_to_pool"] & ~sc_all_data_no_duplicates["is_from_pool"]].copy()

In [87]:
to_pool_obs.rename(columns={"origin_from_address": "liquidity_provider", 
                            "origin_to_address": "manager_address", 
                            "to_address":"pool_address", 
                            "contract_address":"token_address"}, inplace=True)


In [97]:
to_pool_tx_cnt = to_pool_obs.groupby(["pool_address", "manager_address", "week", "liquidity_provider"]).agg(
    sc_input_tx_cnt = ("tx_hash", "nunique"),
).reset_index()

In [98]:
from_pool_obs.rename(columns={"origin_from_address": "liquidity_provider", 
                            "origin_to_address": "manager_address", 
                            "from_address":"pool_address", 
                            "contract_address":"token_address"}, inplace=True)
from_pool_tx_cnt = from_pool_obs.groupby(["pool_address", "manager_address" , "week", "liquidity_provider"]).agg(
    sc_removal_tx_cnt = ("tx_hash", "nunique"),
).reset_index()

In [99]:
from_pool_tx_cnt.to_csv("C:/Users/zxz170017/lp_sc_input.csv", index=False)
to_pool_tx_cnt.to_csv("C:/Users/zxz170017/lp_sc_output.csv", index=False)

In [118]:
neither_to_pool_nor_from_pool["from_lp_to_sc"] = neither_to_pool_nor_from_pool["origin_to_address"].isin(sc_verification_data_unique_managers)
part_two_useful = neither_to_pool_nor_from_pool[neither_to_pool_nor_from_pool["from_lp_to_sc"]].copy()
part_two_useful["from_lp_input"] = part_two_useful["to_address"] == part_two_useful["origin_to_address"]
part_two_useful["from_lp_output"] = part_two_useful["from_address"] == part_two_useful["origin_to_address"]

In [119]:
part_two_useful.rename(columns={"origin_from_address": "liquidity_provider", 
                            "origin_to_address": "manager_address", 
}, inplace=True)

In [121]:
part_two_useful_lp_input = part_two_useful[part_two_useful["from_lp_input"]].copy()
part_two_useful_lp_output = part_two_useful[part_two_useful["from_lp_output"]].copy()

In [122]:
part_two_tx_cnt_input = part_two_useful_lp_input.groupby(["manager_address" , "week", "liquidity_provider"]).agg(
    sc_input_tx_cnt_part_two = ("tx_hash", "nunique"),
).reset_index()
part_two_tx_cnt_ouput = part_two_useful_lp_output.groupby(["manager_address" , "week", "liquidity_provider"]).agg(
    sc_removal_tx_cnt_part_two = ("tx_hash", "nunique"),
).reset_index()

In [125]:
part_two_tx_cnt_input.to_csv("C:/Users/zxz170017/lp_sc_input_part2.csv", index=False)
part_two_tx_cnt_ouput.to_csv("C:/Users/zxz170017/lp_sc_output_part2.csv", index=False)